# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Achieving code reuse in F#”

>LINQ and C# 3 introduced a whole raft of features that were inspired by the world of functional programming. One of the biggest takeaways was the pervasive use of higher order functions (HOF) throughout the LINQ framework. Despite the somewhat technical name, a higher-order function is a function that takes another function as one of its arguments.

In the world of C#, the use of `Func<>` is the most expressive and succinct way to achieve code reuse through higher-order functions:


In [ ]:
public class Address
{
    public string Street { get; set; }
    public string Town { get; set; }
    public string City { get; set; }
}

public class Customer
{
    public string Forename { get; set; }
    public string Surname { get; set; }
    public int Age { get; set; }
    public Address Address { get; set; }
    public string EmailAddress { get; set; }
}

public static IEnumerable<Customer> GetCustomersWithInValidAddress(this IEnumerable<Customer> data,
    Func<Address, bool?> addressValidator)
{
    if(data == null) return Enumerable.Empty<Customer>();
    return data.Where(i => !i.HasValidAddress(addressValidator).GetValueOrDefault());
}

public static bool? HasValidAddress(this Customer data,
    Func<Address, bool?> addressValidator)
{
    return addressValidator?.Invoke(data?.Address);
}


The `addressValidator` [strategy](https://en.wikipedia.org/wiki/Strategy_pattern) passed to the `HasValidAddress` extension method (which cannot be in a `static class` for the scripting version of C#) is effectively a function, making `HasValidAddress` act like a higher-order function. With this HOF in place, `GetCustomersWithInValidAddress` builds on it with LINQ. We can then find the Customers with an invalid Address:


In [ ]:
var customers = new[]
{
    new Customer
    {
        Forename = "Abe",
        Surname = "Sinceros",
        Age = 32,
        EmailAddress = "abe@me.com",
        Address = new Address { Street = "invalid", Town = null, City = null }
    },
    new Customer
    {
        Forename = "Betsy",
        Surname = "Roos",
        Age = 37,
        EmailAddress = "broos@microsoft.com",
        Address = new Address { Street = "123 Elm Street", Town = "Elmore", City = "Elmore Parish" }
    },
    new Customer
    {
        Forename = "Gilfraxian",
        Surname = "Stalag",
        Age = 39,
        EmailAddress = "gs@apple.com",
        Address = new Address { Street = "4333 Forest Circle", Town = null, City = "Sangelos" }
    },
};

Func<Address, bool?> addressValidator = address =>
{
    if(address == null) return false;
    if(string.IsNullOrWhiteSpace(address.Street)) return false;
    if(string.IsNullOrWhiteSpace(address.City)) return false;

    return true;
};

customers.GetCustomersWithInValidAddress(addressValidator)

index Forename Surname Age Address EmailAddress 0 Abe Sinceros 32 Street Town City invalid <null> <null> abe@me.com

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about